In [16]:
import random
from collections import Counter

# Mo' Cards, Mo' Problems

We can continue to build towards a functioning poker game here, and work to add additional features and logic bit by bit. Our starting point will be where we finished last time, with working Card and Deck classes that can be used to create and deal out cards. My first goals here are to:
<ul> 
<li> Extend the Deck class into the Hand class. The Hand is meant to represent the cards held by a player, and will be used to determine the winner of a round of poker. To acheive this we need to add scoring logic to the Hand class. </li>
<li> Create a basic Game class that will "run" the game. This should be able to create the Deck, deal Hands, give them to Players, and determine winners. As we add more logic, this will basically be the "dealer" or "manager" of the game and hold all the stuff for a game. This will probably require that we periodically make changes to what's in some of the other classes, as the way things work together will change a bit as we reform it into an actual game. </li>
</ul>

I'll try to get that framework working today, even if the internals are not fully functional or complete. If we have working scoring for some poker hands we can use that to build the rest, then go back and make our scoring logic handle more complex and obscure things later. 

### Deck Constructor

One change that we will need to make is to the Deck class constructor. In particular, we want to be able to create a deck in two ways:
<ul>
<li> Make it from scratch, with 52 cards optionally shuffled. </li>
<li> Make it from some cards. </li>
</ul>

We want to add the second option, so we can make new decks as well as "sub-decks" from existing decks or cards. This will also be useful to make Hands later, as they are basically a sub-deck. To do so I'm going to just branch the constructor - if the populate option is set, then I'll fill the deck as new; if we provide an *args of cards, then we'll make an empty deck and add those cards to it. This logic can likely be made a little cleaner, but we can worry about that later. In the future we may want to make some changes here to the Deck class overall if we want it to be flexible and adaptable to different games. 

In [17]:
class Card():
    
    suits = ['Hearts', 'Diamonds', 'Clubs', 'Spades']
    rank = ['Two', 'Three', 'Four', 'Five', 'Six', 'Seven', 'Eight',
                'Nine', 'Ten', 'Jack', 'Queen', 'King', 'Ace']

    def __init__(self, suit, rank):
        self._suit = suit
        self._rank = rank
    def __str__(self):
        ret_string = "{} of {}".format(self.rank, self.suit)
        return ret_string
    @property
    def suit(self):
        return self._suit
    @property
    def rank(self):
        return self._rank
    # Comparison Operators
    def __lt__(self, other):
        self_rank =  Card.rank.index(self.rank)
        other_rank = Card.rank.index(other.rank)
        if self_rank == other_rank:
            self_suit = Card.suits.index(self.suit)
            other_suit = Card.suits.index(other.suit)
            return self_suit < other_suit
        return self_rank < other_rank
    def __gt__(self, other):
        return other.__lt__(self)
    
    def __eq__(self, other):
        return (self.suit == other.suit) and (self.rank == other.rank)
    
class Deck():

    # Card Information
    suits = ['Hearts', 'Diamonds', 'Clubs', 'Spades']
    rank = ['Two', 'Three', 'Four', 'Five', 'Six', 'Seven', 'Eight',
                'Nine', 'Ten', 'Jack', 'Queen', 'King', 'Ace']
    
    # This stuff is to make a function to convert to short to the short version. 
    # This doesn't really matter function-wise
    short_suits = ['H', 'D', 'C', 'S']
    short_rank = ['2', '3', '4', '5', '6', '7', '8', '9', '10',
                  'J', 'Q', 'K', 'A']
    @staticmethod
    def tooShort(card):
        return Deck.short_rank[Deck.rank.index(card.rank)]+Deck.short_suits[Deck.suits.index(card.suit)]
    
    def __init__(self, *cards, shuffle=True, populate=False):
        self._deck = []
        
        if populate:
            self.populate52()
        elif len(cards) > 0:
            for card in cards:
                self.deck.append(card)
        if shuffle:
            self.shuffle()
    @property
    def deck(self):
        return self._deck
    def shuffle(self):
        random.shuffle(self.deck)
    def populate52(self):
        self._deck = []
        for suit in self.suits:
            for value in self.rank:
                self.deck.append(Card(suit, value))
    
    # Override some functions and operators
    def __str__(self):
        return_string = ""
        for i, card in enumerate(self.deck):
            return_string += str(i)+": "+str(card)+"\n"
        return return_string
    def __iter__(self):
        return self
    def __next__(self):
        try:
            return self.deck.__next__()
        except:
            raise StopIteration
    def __len__(self):
        return len(self.deck)
    
    # Deal Cards. Make sure that the cards leave the deck. 
    # Rerutn the hands as a list of Decks
    def deal(self, num_hands=1, card_per_hand=1):
        hands = []
        for i in range(num_hands):
            hand = []
            for j in range(card_per_hand):
                hand.append(self.deck.pop())
            tmp = Deck(*hand, shuffle=False, populate=False)
            #print(hand)
            #print(tmp)
            hands.append(tmp)
        return hands
    
    # Add/Remove Cards
    def addCard(self, card):
        self.deck.append(card)
    def removeCard(self, suit, rank):
        for i, c in enumerate(self.deck):
            if c == Card(suit, rank):
                return self.deck.pop(i)

#### Testing - Deck Constructor and Dealing

We should test that the deck can deal. Some hints are on the last notebook on what I did to test...

## Hand Class

To play a game we need a class for the Hand, that we can inherit from the Deck class. The primary thing we need to add to the Hand class to differentiate it from the Deck class is the ability to score itself. Each hand should be able to compute its value and compare to other hands, so we can determine winners. 

### Scoring Hands

Scoring hands is something that we can likely implement in many ways. My method is going to be pretty simple, and I'm going to build it incrementally:
<ul>
<li> Create a checkHands method that will check for whichever hands we have made the logic for, and return a value that we can use to compare. </li>
<li> Create a method for each hand that we want to check for - pairs, straights, flushes, etc. </li>
<li> In the __lt__ comparison, we can compare those values for sorting. </li>
</ul>

This will allow us to make a scoring system that works and is more or less functional, just incomplete. I know that there are still several things I need to add to make it work correctly, but this starting point will work well enough for me to be able to test it out and develop the rest of the logic. In particular, I know I'm going to need to modify things to add:
<ul>
<li> The rest of the hands that need to be checked. </li>
<li> The ability to look at tie breakers, such as if two hands have the same pair, which has the higher kicker. </li>
</ul>

One thing I'm going to use to make some of the hand checking logic easier is a Counter. A Counter is a subclass of the dictionary class that is designed to count the number of times each element appears in a list. This will make it easy to check for pairs, three of a kind, and four of a kind.

In [18]:
class Hand(Deck):
    hands = ['High Card', 'Pair', 'Two Pair', 'Trips', 'Straight', 'Flush',
             'Full House', 'Quads', 'Straight Flush', 'Royal Flush']
    
    # Converts a deck to a hand
    @staticmethod
    def deckToHand(deck):
        return Hand(*deck.deck)

    def __init__(self, *cards, size=5):
        super().__init__(*cards)
        self.size = size

    # Check different hands
    # This is not complete or perfect!!! 
    def checkFlush(self):
        suit = self.deck[0].suit
        for card in self.deck:
            if card.suit != suit:
                return False
        return True
    def checkPair(self):
        c = Counter([card.rank for card in self.deck])
        if 2 in c.values():
            return True
        return False
    def checkTrips(self):
        c = Counter([card.rank for card in self.deck])
        if 3 in c.values():
            return True
        return False
    def checkQuads(self):
        c = Counter([card.rank for card in self.deck])
        if 4 in c.values():
            return True
        return False
    def checkTwoPair(self):
        c = Counter([card.rank for card in self.deck])
        vals = c.values()
        val_counter = Counter(vals)
        if 2 in val_counter.values():
            return True
    def checkFullHouse(self):
        return self.checkTrips() and self.checkPair()
    
    # Check all the hands that we score and define a score
    def checkHand(self):
        if self.checkQuads():
            return 7
        elif self.checkFullHouse():
            return 6
        elif self.checkFlush():
            return 5
        elif self.checkTrips():
            return 4
        elif self.checkTwoPair():
            return 3
        elif self.checkPair():
            return 2
        elif self.checkPair():
            return 1
        else:
            return 0

    
    def __lt__(self, other):
        self_score = self.checkHand()
        other_score = other.checkHand()
        if self_score < other_score:
            return True
        elif self_score > other_score:
            return False

#### Testing - Hands

We can test hands here a bit. The primary concern is the scoring, as most of the rest of the logic is already tested in the Deck class. We can test the scoring by making a few hands and comparing them. 

## To Go... Make a Game

We can try our simple setup by making a game of poker, again, we can start with a simple dumb version and revise as we get better. 

The eventual goal of the Game class is to be able to play a game of poker:
</ul>
<li> Deal hands. </li>
<li> Take bets. </li>
<li> Exchange cards. </li>
<li> Take more bets. </li>
</ul>

This is relatively involved, particularly with multiple rounds of betting and exchanging cards. I think that we should simplify this a bit - what I'll do here is a choice that I made based on an estimation of the difficulty to implement and the "usefulness" of the steps, this isn't a specific rule that's being followed, it's an educated guess. 
<b>
<ul>
<li> Deal hands. </li>
<li> Check for a winner. </li>
</ul>
</b>

In the midst of this, I will also need to think of other factors that will be important. One clear thing we'll need is a player. If we think about what we need to track for a player, we'll need to know their hand, their money, and their bets. As with everything else, I can break this down - fist off I won't be dealing with bets, so each player will just be a hand of cards held in a list. If I want to add betting later I can then either do something like make each player a tuple of money and hand, or I can make a player class that represents the player. I will eventually need a player class, but if it feels complex for me, I can build other things and hold that off for a bit. 

Then add in one of the complicating steps, either betting or exchanging cards. First though, I want a basic game. 

In [19]:
class FiveCardDraw():

    def __init__(self, num_players=4, num_cards=5):
        self.num_players = num_players
        self.num_cards = num_cards
        self.deck = Deck(shuffle=True, populate=True)
        self.hands = list(map(Hand.deckToHand, self.deck.deal(num_hands=self.num_players, card_per_hand=self.num_cards)))
    
    def __str__(self):
        return_string = ""
        for i, hand in enumerate(self.hands):
            return_string += "Player: "+str(i)+":\n"+str(hand)+"\n"
        return return_string
    
    def calculateWinner(self, to_print=False):
        scores = []
        for hand in self.hands:
            scores.append(hand.checkHand())
        if to_print:
            print(scores)
        return scores.index(max(scores))

#### Testing the Game

We can test the game to this point. We expect it to:
<ol>
<li> Make a game. </li>
<li> Make a deck. </li>
<li> Deal hands. </li>
<li> Print the "game" </li>
<li> Check for a winner. </li>
</ol>

In [20]:
fcd = FiveCardDraw()
print(fcd)

Player: 0:
0: Two of Spades
1: Eight of Clubs
2: King of Diamonds
3: Four of Clubs
4: Seven of Clubs

Player: 1:
0: Nine of Diamonds
1: Four of Hearts
2: Seven of Diamonds
3: Ten of Diamonds
4: Ace of Spades

Player: 2:
0: King of Spades
1: Two of Diamonds
2: King of Clubs
3: Five of Hearts
4: Queen of Hearts

Player: 3:
0: Queen of Spades
1: Nine of Clubs
2: Six of Spades
3: Jack of Clubs
4: Six of Clubs




In [21]:
fcd.calculateWinner(to_print=True)

[0, 0, 2, 2]


2

## Play on Playa'

Lastly, for now, I can add in a Player class to my game. Above, the hands are just kept in a list. When we have betting and we are tabulating banks it likely makes much more sense to have a player that holds the bank account as well as their given hand. Like with everything else, I can start simple and add complexity as I go. To get started, I think I need:
<ul>
<li> The ability to hold a hand of cards. </li>
<li> A bank account. </li>
<li> The ability to add and subtract money from the bank. </li>
<li> Probably - a way to sort players based on their hands. This isn't inherently necessary, but rather than pulling the hand from every player and then sorting them, it's probably easier to just have a method that sorts the players based on their hands. </li>
</ul>

We also need to modify the game class a little to deal with the new Player objects. I didn't have to make many changes, as the only thing that really changed is that now instead of each player being represented by a Hand in a list, each one is now represented by a Player object in a list. Our player can still just pass much of the logic down to its hand, like for sorting values and even most of the printing stuff. One change that I can think of is that the logic in starting the game needs to change - above the game generated hands in the constructor. In a real game, our players are created, then persist for many hands, so we probably want to put all the logic for dealing and scoring outside the constructor, as we want to call it over and over. 

<b>Note:</b> for simple interactive things like these games, a common way to make them "play" is to have a loop that goes until there is some quit signal, like clicking a button or entering some "end" value. That loop will basically be "play_game" over and over until we manually exit it. 

In [22]:
class Player():

    def __init__(self, name, bank=1000):
        self.name = name
        self.hand = None
        self._bank = bank
    
    def setHand(self, hand):
        self.hand = hand

    def __str__(self):
        return self.name+" - : "+str(self._bank)+"\n"+str(self.hand)
    
    def __lt__(self, other):
        return self.hand.__lt__(other.hand)
    def checkHand(self):
        return self.hand.checkHand()
    
    def getBank(self):
        return self._bank
    def setBank(self, value):
        self._bank = value
    def addBank(self, value):
        self._bank += value

In [23]:
class FiveCardDraw():

    def __init__(self, num_players=4, num_cards=5, start_bank=1000):
        self.num_players = num_players
        self.num_cards = num_cards
        self.players = []

        for i in range(num_players):
            self.players.append(Player("Player "+str(i), start_bank))
    
    def __str__(self):
        return_string = ""
        for i, hand in enumerate(self.players):
            return_string += "Player: "+str(i)+":\n"+str(hand)+"\n"
        return return_string
    
    def calculateWinner(self, to_print=False):
        scores = []
        for hand in self.players:
            scores.append(hand.checkHand())
        if to_print:
            print(scores)
        return scores.index(max(scores))
    
    def playHand(self):
        deck = Deck(shuffle=True, populate=True)
        hands = list(map(Hand.deckToHand, deck.deal(num_hands=self.num_players, card_per_hand=self.num_cards)))
        
        for i, hand in enumerate(hands):
            self.players[i].setHand(hand)
        
        winner = self.calculateWinner()
        return winner, hands[winner]


#### Testing the New Game

We can test the game to this point. We expect it to:
<ol>
<li> Make a game. </li>
<li> Make players. </li>
<li> Deal and play, all in one step here. </li>
</ol>

In [24]:
fcd = FiveCardDraw()
print(fcd)

Player: 0:
Player 0 - : 1000
None
Player: 1:
Player 1 - : 1000
None
Player: 2:
Player 2 - : 1000
None
Player: 3:
Player 3 - : 1000
None



In [25]:
winner, win_hand = fcd.playHand()
print("Winner: ", winner)
print(win_hand)
print(fcd.calculateWinner(to_print=True))

Winner:  1
0: Six of Diamonds
1: Ace of Spades
2: Six of Spades
3: Five of Diamonds
4: Five of Hearts

[2, 3, 0, 0]
1


## Keep it Going

We have several things that we can do now to make things improve and get closer to poker, among them:
<ul>
<li> Add betting. </li>
<li> Complete the scoring logic. </li>
<li> Add exchanging cards. </li>
</ul>

Each of these things is somewhat independent of each other, and we can pursue them in pretty much any order. I think that the items are ordered roughly in order of complexity in the list above, so I'll start with betting next time...

### Import Me!

As we are editing here, we keep needing to re-paste the code in each notebook. This is annoying and dumb. Next time we pick this up we can move this code into a .py file, then import it to a notebook to play with it. This will be way more convenient going forward. 

### Some Things Don't Make Sense

As I'm making this, there are also some things emerging that I know I might want to change. I don't need to tackle them all right now though, as things work OK. It is pretty common for my idea of what needs to be made and how to do it to evolve as we work through the problem and figure out how to do things. For example, I probably want to make the Deck able to be inherited from, so that I can make a Deck for a different game, though I haven't thought through this yet. I also have the list of card ranks in more than one place now, I probably want to make that exist in only one place. As well, there's no tiebreaker logic in the scoring, and that needs special consideration, but can wait...

It is ok to do things in a way that works for now, even if we need to change it later. We don't need to solve every single challenge and problem up front before things work, because that can be really hard to do. We can make a trash version that grows to handle a more and more complete version of what we want. 